# Práctica 2: Modelo cinemático inverso de un manipulador y planteamiento de trayectoria

## Objetivo

El objetivo de esta práctica es que el alumno comprenda, interprete y modifique la información básica de los formatos de Universal Robot Description File (URDF).


### Metas 

- Que el alumno aplique un modelo cinemático inverso para calcular una trayectoria a partir de una posición actual hacia una posición final
- Que el alumno grafique y analice los resultados del modelo

### Contribución al perfil del egresado

La siguiente práctica contribuye en los siguientes puntos al perfil del egresado:

#### Aptitudes y habilidades

- Para modelar, simular e interpretar el comportamiento de los sistemas mecatrónicos.
- Para diseñar, construir, operar y mantener los sistemas mecatrónicos y sus componentes.

#### Actitudes

- Tener confianza en su preparación académica.
- Comprometido con su actualización, superación y competencia profesional.

#### De tipo social

- Promover el cambio en la mentalidad frente a la competitividad internacional.

## Rúbrica de evaluación

La evaluación de la práctica contará de los siguientes puntos y se evaluará con los siguientes criterios:

| Elemento | Porcentaje |
| ------:| -----------:|
| **Cuestionario previo** | 15% | 
| **Desarrollo** | 35% |
| **Análisis de resultados**  | 35% |
| **Conclusiones** | 15% |

<br>


| Elemento | Malo | Regular | Bueno |
| ------:| ------ | --------| ------|
| **Cuestionario previo** | El trabajo no contiene cuestionario previo o todas las preguntas son incorrectas (0%)| Al menos la mitad de las preguntas son correctas (8%) |  Todas las preguntas son correctas (15%) |
| **Desarrollo** | El trabajo no contiene desarrollo o su planteamiento no concuerda con lo deseado (0%) | El desarrollo está mal planteado o no llega a los resultados esperados (10%) | El desarrollo tiene un planteamiento adecuado y llega a los resultados esperados (35%) |
| **Análisis de resultados**  | El trabajo no contiene análisis de resultados o la información no se está interpretando correctamente (0%) | La interpretación de los resultados es parcial o desorganizada (10%) | Realiza un correcto análisis de los resultados de forma organizada   (35%) |
| **Conclusiones** | El trabajo no contiene conclusiones o no hacen referencia al trabajo desarrollado y los objetivos planteados (0%) | La redacción de las conclusiones es desorganizada o confusa (8%) | Las conclusiones del trabajo son claras y hacen referencia al trabajo desarrollado y los objetivos planteados (15%) | 



## Introducción

### Transformaciones homogéneas
Las transformaciones homogéneas permiten hacer el planteamiento del modelo cinemático de un robot, considerando las posiciones y orientaciones de las juntas del robot respecto al sistema de referencia de una junta anterior

Este planteamiento es el **modelo de cinemática directa**, que nos permite obtener la posición y velocidad del efector final de un manipulador en términos de los valores de la posición y velocidad de sus juntas (espacio de trabajo)

A través de este modelo se puede obtener el **modelo de cinemática inversa**, que permite obtener la velocidad de las juntas de un robot a partir de la velocidad deseada del efector final.

### Planteamiento de una trayectoria
Si se conoce el punto inicial y final de una trayectoria deseada, se pueden obtener los puntos intermedios de la trayectoria. La forma más fácil de realizar esta interpolación es a través de un spline. El orden del spline permitirá controlar las condiciones inicial y final de la posición, velocidad ó aceleración que tendrá el efector final durante el trayecto. 

Juntando la interpolación de la trayectoria y el modelo de la cinemática inversa, se pueden obtener todos los puntos intermedios de la trayectoria que deben seguir las juntas del robot para que el efector final siga una trayectoria.

## Cuestionario previo

Responder de forma breve las siguientes preguntas:

### ¿Qué son las transformaciones homogéneas?

Las **transformaciones homogéneas** se definen como matrices de dimensión **4×4** que permiten representar la transformación de un vector entre distintos sistemas de coordenadas. Estas matrices se utilizan principalmente en robótica, visión por computadora y gráficos 3D.

### Componentes de una matriz homogénea:

Una matriz homogénea está compuesta por los siguientes elementos:

- 🔁 **Rotación (3×3)**: Representa la orientación del sistema de coordenadas.
- ➕ **Traslación (3×1)**: Indica el desplazamiento respecto a otro sistema de coordenadas.
- 📐 **Escalamiento (1×1)**: Factor de escala aplicado (usualmente 1 en transformaciones rígidas).
- 🔄 **Perspectiva (1×3)**: Se usa principalmente en gráficos 3D, rara vez en robótica.

### ¿Para qué se utilizan?

- ✅ Representar la **posición y orientación** de un sistema girado y trasladado respecto a un sistema fijo.
- 🔄 Transformar un **vector expresado en coordenadas móviles** a su representación en un sistema de referencia fijo.
- 🌀 **Rotar y trasladar** un vector con respecto a un sistema fijo.


### ¿Que nos permite obtener el modelo de cinemática directa de un manipulador?
### **Modelo Cinemático Directo (MCD) de un Robot**

## **Pasos para obtener el MCD**

### 1. **Asignación de sistemas de coordenadas**  
   - Utilizar la **convención Denavit-Hartenberg (DH)** para asignar marcos de referencia a cada eslabón.  
   - **Ejes**:  
     - \( Z_i \): Alineado con el eje de la articulación \( i+1 \).  
     - \( X_i \): Perpendicular común entre \( Z_{i-1} \) y \( Z_i \).  

### 2. **Parámetros Denavit-Hartenberg**  
   Para cada articulación \( i \), definir:  
   - \( \theta_i \): Ángulo de la articulación (rotacional).  
   - \( d_i \): Desplazamiento lineal (prismática).  
   - \( a_i \): Longitud del eslabón (distancia entre ejes \( Z \)).  
   - \( \alpha_i \): Ángulo de torsión (ángulo entre \( Z_{i-1} \) y \( Z_i \)).  

### 3. **Matrices de transformación homogénea**  
   - Cada matriz \( A_i \) relaciona el marco \( i \) con \( i-1 \):  
     \[
     A_i = \text{Rot}(Z, \theta_i) \cdot \text{Trans}(Z, d_i) \cdot \text{Trans}(X, a_i) \cdot \text{Rot}(X, \alpha_i)
     \]  
   - Forma general de \( A_i \):  
     \[
     A_i = \begin{bmatrix}
     \cos\theta_i & -\sin\theta_i \cos\alpha_i & \sin\theta_i \sin\alpha_i & a_i \cos\theta_i \\
     \sin\theta_i & \cos\theta_i \cos\alpha_i & -\cos\theta_i \sin\alpha_i & a_i \sin\theta_i \\
     0 & \sin\alpha_i & \cos\alpha_i & d_i \\
     0 & 0 & 0 & 1
     \end{bmatrix}
     \]

### 4. **Transformación total**  
   - Multiplicar todas las matrices \( A_i \):  
     \[
     T = A_1 \cdot A_2 \cdot \ldots \cdot A_n
     \]  
   - \( T \) es una matriz \( 4 \times 4 \) que contiene:  
     - **Orientación**: Submatriz superior \( 3 \times 3 \).  
     - **Posición**: Vector \( 3 \times 1 \) en la última columna.  

### 5. **Extracción de posición y orientación**  
   - **Posición**: \( (x, y, z) = [T_{14}, T_{24}, T_{34}] \).  
   - **Orientación**: Usar ángulos de Euler o cuaterniones según sea necesario.  


- ¿Que nos permite obtener el modelo de cinemática inversa de un manipulador?
>Respuesta

- ¿De que formas se puede interpolar la trayectoria de un efector final entre dos puntos?
>Respuesta

## Desarrollo

### 1. Planteamiento de la cinemática directa
En esta primera parte, se crearán las transformaciones homogéneas y el modelo de cinemática directa de un robot RRR, incluyendo la matriz del Jacobiano. Se recomienda usar **Sympy** para el planteamiento de las expresiones. 
Un diagrama del robot se muestra en la imagen:

<img src="images/p2_1.png" alt = "Robot RRR" width="300" height="300" display= "block"/>

** Considerar valores cualesquiera para las dimensiones de los eslabones y la posición inicial de las juntas

In [ ]:
# Colocar y correr el código dentro de estas celdas. 
# Agregar cuantas consideren conveniente

### 2. Planteamiento de la trayectoria

En esta segunda parte, se planteará el código que permita definir los puntos intermedios de una trayectoria, la cual debe tener velocidades y aceleraciones nulas al inicio y al final. Se deben incluir también las gráficas de la posición, velocidad y aceleración del efector final. 

Calcular la trayectoria considerando de forma general tiempo de duración, puntos inicial y final, y con una tasa de muestreo de 30 muestras por segundo. 

In [ ]:
# Colocar y correr el código dentro de estas celdas. 
# Agregar cuantas consideren conveniente

### 3. Cinemática inversa
A partir del modelo de la cinemática directa, obtener la expresión e la cinemática inversa, que relacione las velocidades de las juntas del robot con la velocidad del efector final. Ya que el modelo de cinemática inversa sólo permite obtener velocidades, obtener también expresiones que permitan obtener la posición de las juntas y sus aceleraciones

In [ ]:
# Colocar y correr el código dentro de estas celdas. 
# Agregar cuantas consideren conveniente

### 4. Aplicación de la cinemática inversa
Finalmente, a partir de los puntos de la trayectoria y el modelo de cinemática inversa, obtener las posiciones, velocidades y aceleraciones de las juntas del robot, así como sus gráficas en función del tiempo

In [ ]:
# Colocar y correr el código dentro de estas celdas. 
# Agregar cuantas consideren conveniente

### 5. Aplicación de la cinemática inversa
Para terminar, subir los archivos de la práctica al repositorio de github

## Análisis de resultados

- ¿Qué utilidad tuvo usar el modelo de cinemática inversa de un robot?
> Respuesta

- ¿Qué utilidad tiene el modelo de cinemática inversa de un robot?
> Respuesta

- ¿Qué utilidad tiene el modelo de cinemática inversa de un robot?
> Respuesta




## Conclusiones

> Respuesta

## Bibliografía 

> En caso de usarse, se deben hacer referencia a la información implementada en formato ieee


